In [8]:
# libraries
import psycopg2 as pg
import pandas as pd
import numpy as np
import os
import psycopg2.extras

# set environment variable for psycog2 (for some systems)
os.environ["PGGSSENCMODE"] = "disable"

# get the stored passwords
f=open("credentials_david.txt", "rt")
pwd=f.readline().strip()  #di   password
hpwd=f.readline().strip() #home password
f.close()

In [2]:
# di test: WORKING
#host_di = "appserver-01.alunos.di.fc.ul.pt"
#db_di = "tpd012"
#user_di = "tpd012"

#conn = pg.connect(host = host_di,
#                  database = db_di,
#                  user = user_di,
#                  password = pwd)
#conn.close()

# local test: WORKING
host_local = "localhost"
db_local = "tpd012"
user_local = "postgres"

conn = pg.connect(host = host_local,
                  database = db_local,
                  user = user_local,
                  password=hpwd)
conn.close()

In [3]:
# functions
def run_sql_command(sql, host, database, user, password):
    """Executes a single SQL statement from a string variable and the database credentials"""
    conn = pg.connect(host = host,
                      database = database,
                      user = user,
                      password = password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close() 

In [4]:
def intTryParse(value):
    """Parse a string to an integer"""
    try:
        a = int(value)
        return True
    except ValueError:
        return False

In [ ]:
listings_al_file_path = '../data/listings_al.csv'
df_listings_al = pd.read_csv(listings_al_file_path)

## 2.2. Preprocessing

The defined _'Property'_ dimension table in __Phase I__ is as follows:

<img src="property_schema.png" width="150" align="center"/>

# __CHANGE SCHEMA__

<img src="PropertyETL.png" align="center"/>

In [ ]:
def delete_null_rows(df,columns):
    """Removes all records with any empty cells from input DataFrame"""
    processed_df = df[columns].copy()
    total_rows = processed_df.shape[0]
    delete_rows = []
    
    if processed_df.isnull().values.any(): # if there are any null values in DataFrame, process DataFrame
        for index, row in enumerate(processed_df.itertuples(), start = 0):
            if (pd.Series(row).isnull().values.any()): # if row has any null value
                delete_rows.append(index) # add row index to delete list
    
    processed_df.drop(df.index[delete_rows], inplace = True) # delete rows fr
    processed_df = processed_df.reset_index().drop('index', axis = 1)
    print('DataFrame contains {} rows. Deleted {} rows ({}% of total rows)'.format(processed_df.shape[0], len(delete_rows), round(len(delete_rows)*100/total_rows, 2)))
    return processed_df

In [ ]:
df_columns = ['id',
              'property_type',
              'room_type',
              'accommodates',
              'bathrooms',
              'bedrooms',
              'beds',
              'bed_type']

df_property = delete_null_rows(df_listings_al,df_columns)

From `df_property`, we create `df_conv_fact`, which will be the table containing a record for each fact ID converted to the dimension format. `property_dimension` will be the _de facto_ property dimension from which we can correspond each fact ID record to the appropriate dimension foreign key according to its set of attributes.

### _'property_type_category'_

We start by creating _'property_type_category'_, a set of four categories from _'property_type'_.

In [ ]:
def get_property_type_category(value):
# change 'property_type' to four categories, exclude exceptions
    if value in ["Apartment", "Serviced apartment", "Aparthotel","Loft"]:
        return 'Apartment'
    if value in ["House","Townhouse","Villa","Dome house","Vacation home","Lighthouse","Casa particular (Cuba)","Tiny house","Farm stay","Cottage"]:
        return 'House'
    if value in ["Guesthouse","Guest suite"]:
        return 'Guesthouse'
    if value in ["Hostel","Bed and breakfast","Boutique hotel","Hotel"]:
        return 'Hotel/Hostel'
    return None # for the case of missing or invalid values

### _'room_type'_

This attribute is already defined in satisfactory categories.

### _'accommodates'_

In [ ]:
def get_accommodates(value):
# change 'property_type' to four categories, exclude exceptions
    if value in [1,2]: return 'Up to 2 guests'
    if value in [3,4]: return 'Up to 4 guests'
    if value in [5,6]: return 'Up to 6 guests'
    return 'Up to 7 guests or more' # no missing values in this column

### _'bathrooms'_

In [ ]:
def get_bathrooms(value):
# change 'bathrooms' to five categories, exclude exceptions
    if value < 0.5: return 'No bathrooms'
    if value < 1.5: return '1 bathroom'
    if value < 2.5: return '2 bathrooms'
    if value < 3.5: return '3 bathrooms'
    if value >= 3.5: return '4+ bathrooms'
    return None # for the case of missing or invalid values

### _'bedrooms'_

In [ ]:
def get_bedrooms(value):
# change 'bedrooms' to five categories, exclude exceptions
    if value == 0: return 'T0'
    if value == 1: return 'T1'
    if value == 2: return 'T2'
    if value == 3: return 'T3'
    if value >= 4: return 'T4+'
    return None # for the case of missing or invalid values

### _'beds'_

In [ ]:
def get_beds(value):
# change 'beds' to five categories, exclude exceptions
    if value == 0: return 'No beds'
    if value == 1: return '1 bed'
    if value == 2: return '2 beds'
    if value == 3: return '3 beds'
    if value >= 4: return '4+ beds'
    return None # for the case of missing or invalid values

## 2.3. Creating the dimension table

In [ ]:
dimension = {}

dimension['ID'] = [value for value in df_property['id']]
dimension['property_type_category'] = [get_property_type_category(value) for value in df_property['property_type']]
dimension['property_type'] = [value for value in df_property['property_type']]
dimension['room_type'] = [value for value in df_property['room_type']]
dimension['accommodates'] = [get_accommodates(value) for value in df_property['accommodates']]
dimension['bathrooms'] = [get_bathrooms(value) for value in df_property['bathrooms']]
dimension['bedrooms'] = [get_bedrooms(value) for value in df_property['bedrooms']]
dimension['beds'] = [get_beds(value) for value in df_property['beds']]
dimension['bed_type'] = [value for value in df_property['bed_type']]

In [ ]:
df_conv_facts = pd.DataFrame(dimension)
df_conv_facts.index += 1
df_conv_facts

In [ ]:
df_conv_facts = delete_null_rows(df_conv_facts,df_conv_facts.columns)

In [ ]:
# create df_property_dimension
property_dimension = df_conv_facts[['property_type_category','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type']].drop_duplicates().copy()
property_dimension = property_dimension.reset_index().drop('index', axis = 1)
property_dimension.index += 1

property_dimension

## 2.4. Attributing dimension keys to facts

Having our dimension DataFrame `property_dimension` ready, we can now use our converted listings table, `df_conv_facts` (which contains Property dimension attributes for all valid facts), to assign each fact its corresponding Property dimension foreign key.

<img src="PropertyETL2.png" align="center"/>

In [ ]:
# change df_property_dimension to property dimension ID
FK = []
for i, row in enumerate(df_conv_facts.itertuples(), start = 1): 
    row = list(row)
    for j, dim_row in enumerate(property_dimension.itertuples(), start = 1):
        dim_row = list(dim_row) 
        if (row[2:] == dim_row[1:]): # if fact attributes are equal to dimension record
            FK.append(j)
            break # leaves nested loop if match is found

In [ ]:
df_conv_facts['Property'] = FK

In [ ]:
df_conv_facts = df_conv_facts[['ID','Property']]
df_conv_facts.index += 1
df_conv_facts

This table can then be merged with all other corresponding tables for the remaining dimensions to produce each fact

## 2.5. Creating the _Property_ dimension table in SQL

The SQL statement to create the table is as follows:

In [ ]:
delete_table = """
DROP TABLE IF EXISTS Property;
"""

create_table = """
CREATE TABLE Property (
    PROPERTY_ID SERIAL PRIMARY KEY NOT NULL,
    PROPERTY_TYPE_CATEGORY VARCHAR(30) CHECK (PROPERTY_TYPE_CATEGORY in ('Apartment', 'Guesthouse', 'House', 'Hotel/Hostel')) NOT NULL,
    PROPERTY_TYPE VARCHAR(30) CHECK (PROPERTY_TYPE in ('Apartment','Serviced apartment','Aparthotel','Loft','House','Townhouse','Villa','Dome house','Vacation home','Lighthouse','Casa particular (Cuba)','Tiny house','Farm stay','Cottage','Guesthouse','Guest suite','Hostel','Bed and breakfast','Boutique hotel','Hotel')) NOT NULL,
    ROOM_TYPE VARCHAR(30) CHECK (ROOM_TYPE in ('Entire home/apt', 'Private room', 'Hotel room', 'Shared room')) NOT NULL,
    ACCOMMODATES VARCHAR(30) CHECK (ACCOMMODATES in ('Up to 2 guests','Up to 4 guests','Up to 6 guests','Up to 7 guests or more')) NOT NULL,
    BATHROOMS VARCHAR(30) CHECK (BATHROOMS in ('No bathrooms','1 bathroom','2 bathrooms','3 bathrooms','4+ bathrooms')) NOT NULL,
    BEDROOMS VARCHAR(10) CHECK (BEDROOMS in ('T0','T1','T2','T3','T4+')) NOT NULL,
    BEDS VARCHAR(10) CHECK (BEDS in ('No beds','1 bed','2 beds','3 beds','4+ beds')) NOT NULL,
    BED_TYPE VARCHAR(30) CHECK (BED_TYPE in ('Real Bed', 'Pull-out Sofa', 'Futon', 'Couch', 'Airbed')) NOT NULL
)
"""

The `create_table` command contains the integrity constraints essential for modelling the dimension.

In [ ]:
# creating table from scratch

run_sql_command(delete_table, host_local, db_local, user_local, hpwd)
run_sql_command(create_table, host_local, db_local, user_local, hpwd)

In [ ]:
# uploading DataFrame data to dimension in database
from sqlalchemy import create_engine

engine = create_engine("postgres://{user}:{password}@{host}:5432/{database}".format(user = user_local,
                                                                                    password = hpwd,
                                                                                    host = host_local,
                                                                                    database = db_local))
property_dimension.to_sql('property',
                          con = engine,
                          if_exists = 'append',
                          index = True,
                          index_label = 'property_id')

## 2.6. Adding new facts

As described in the pipeline diagram, when new data is obtained, the DataFrame's columns of interest are converted to the _Property_ dimension format are compared with the dimension table for attributes and ID. In this way, 

In [11]:
# function to query table and convert it to pandas dataframe
def query_table(conn, table_name):
    sql = "select * from {};".format(table_name)
    #return dataframe
    return pd.read_sql_query(sql, conn)

# for this function to run, the dataframes must have the same columns, in the same order
def get_data_to_insert(df_etl, df_sql):
    return df_etl[~df_etl.isin(df_sql)].dropna(how = 'all')

# function for bulk insert
def insert_data(df, table_name, conn):
    df_columns = list(df)
    columns = ",".join(df_columns)
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
    insert_stmt = "INSERT INTO {} ({}) {}".format(table_name,columns,values)
    success = True
    try:
        cursor = conn.cursor()
        psycopg2.extras.execute_batch(cursor, insert_stmt, df.values)
        conn.commit()
        success = True
    except pg.DatabaseError as error:
        success = False
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return success

In [13]:
# retrieve Property dimension table
conn = pg.connect(host = host_local,
                        database = db_local,
                        user = user_local,
                        password = hpwd)

property_dimension_sql = query_table(conn, 'property')

conn.close()

property_dimension_sql

TypeError: query_table() got an unexpected keyword argument 'index_col'

In [ ]:
# testing with dummy_data
new_data_file_path = '../data/MOCK_DATA.csv'
df_new_data = pd.read_csv(new_data_file_path)
df_new_data.index += 1

# dummy data is already converted to dimension attributes; that preprocessing is easy to replicate
df_new_data